In [ ]:
import cv2
import numpy as np
import Image
import pytesseract
import string


filename = "o4.jpg"
filepath = "vi_osmo_numbers/color/" + filename
img = cv2.imread(filepath)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# filter for edge
#gray_blur = cv2.bilateralFilter(gray, 11, 17, 17)
gray_blur = cv2.bilateralFilter(gray, 11, 13, 13)
#gray_blur = gray

sigma=0.33
# compute the median of the single channel pixel intensities
v = np.median(gray_blur)
# apply automatic Canny edge detection using the computed median
lower = int(max(0, (1.0 - sigma) * v))
upper = int(min(255, (1.0 + sigma) * v))
edged = cv2.Canny(gray_blur, lower, upper)
#otsu_threshold, edged = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#edged = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 1)
# cv2.imshow("edged", edged)
# cv2.waitKey(0)
#cv2.destroyAllWindows()

# find contours in the edged image, keep only the largest
# ones, and initialize our screen contour
(cimg, cnts, hierarchy) = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#(cimg, cnts, hierarchy) = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
#(cimg, cnts, hierarchy) = cv2.findContours(edged.copy(), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
#cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]
cnts_idxes = sorted(range(len(cnts)), key=lambda k: cv2.contourArea(cnts[k]), 
                    reverse = True)[:10]
#screenCnt = None

bFoundTarget = False
# loop over our contour
#for c, hi in zip(cnts, hierarchy[0]):
for ci in cnts_idxes:
    c = cnts[ci]
    hi = hierarchy[0][ci]

#     # display contours and its area
#     mask = np.zeros(gray.shape,np.uint8)
#     cv2.drawContours(mask,[c],0,255,-1)
#     cv2.imshow(str(ci)+"mask", mask)

    #print cv2.isContourConvex(c)
    #print hi
    #if hi[0] == -1 and hi[1] == -1 and hi[2] == -1 and hi[3] != -1:
    #if hi[0] == -1 and hi[1] == -1 and hi[2] != -1 and hi[3] != -1:
    if hi[3] != -1:
        
#         # draw contours on original image
#         img_copy = img.copy()
#         cv2.drawContours(img_copy, [c], -1, (0, 255, 0), 3)
#         cv2.imshow(str(ci)+"contours", img_copy)
#         #cv2.waitKey(0)

        # approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        if len(approx) == 4:
            continue

        # CROP ONLY THE NUMBER
        x,y,w,h = cv2.boundingRect(c)
        cropped = img.copy()[y:y+h,x:x+w]
        #cropped_gray = gray[y:y+h,x:x+w]
        #cropped = img.copy()[y+5:y+h-5,x+5:x+w-5]
        #cropped_gray = gray[y+5:y+h-5,x+5:x+w-5]
        cv2.imshow("cropped", cropped)

        # check parent part
        parent_idx = hi[3] 
        while parent_idx != -1:
            #print "parent_idx", parent_idx
            # parent contour
            parent_c = cnts[parent_idx]
            parent_hi = hierarchy[0][parent_idx]
            # approximate the contour
            peri = cv2.arcLength(parent_c, True)
            approx = cv2.approxPolyDP(parent_c, 0.11 * peri, True)        
            # if our approximated contour has four points, then
            # we can assume that we have found our screen
            #print "parent approx: ", len(approx)
            x,y,w,h = cv2.boundingRect(parent_c)
            aspect_ratio = w / h
            #print "aspect_ratio of ", parent_idx, aspect_ratio
            if aspect_ratio >=1 and len(approx) == 4:
                #print "aspect_ratio of ", parent_idx, aspect_ratio
                parent_cropped = img.copy()[y:y+h,x:x+w]
                #cropped_gray = gray[y:y+h,x:x+w]
                #cropped = img.copy()[y+5:y+h-5,x+5:x+w-5]
                #cropped_gray = gray[y+5:y+h-5,x+5:x+w-5]                

                # for child - target number
                child_idx = parent_hi[2]
                child_c = cnts[child_idx]
                # draw contours on original image
                img_copy = img.copy()
                cv2.drawContours(img_copy, [child_c], -1, (0, 255, 0), 3)
                cv2.imshow(str(child_idx)+"contours", img_copy)
                cv2.imshow("p"+str(child_idx), parent_cropped)

                bFoundTarget = True
                break
            else:
                parent_idx = parent_hi[3]

        #
        if bFoundTarget:
            break

# Threshold cropped so it becomes binary
#otsu_threshold, cropped_binary = cv2.threshold(cropped_gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
# kernel = np.ones((3, 3), np.uint8)
# cropped_binary = cv2.morphologyEx(cropped_binary, cv2.MORPH_CLOSE, kernel, iterations=2)
# #closing = cv2.bitwise_not(closing)

#cv2.imshow("edged", edged)
#cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 3)
#cv2.drawContours(img, [box], -1, (0, 255, 0), 3)
#cv2.imshow("digit tag", img_copy)
#cv2.imshow("cropped", cropped)
#cv2.imshow("gray", mask_gray)
#cv2.waitKey(0)
#cv2.destroyWindow("gray")
#cv2.imshow("deskewed", mask_deskewed)
#cv2.imshow("cropped_edged", cropped_edged)

cv2.waitKey(0)
cv2.destroyAllWindows()

#horizontal_cropped = cv2.flip( mask, 1 )
#cv2.imwrite(filename + "_cropped.png", cropped)
#cv2.imwrite(filename + "_numbox.png", parent_cropped)       


# # preprocess image for Tesseract 4        
# #horizontal_cropped = cv2.flip( parent_cropped, 1 )
# horizontal_cropped = parent_cropped
# rgb_im = cv2.cvtColor(horizontal_cropped, cv2.COLOR_BGR2RGB)
# pil_im = Image.fromarray(rgb_im)
# pil_im.info["dpi"] = (300, 300)
# pil_im.save("pil_cropped.png")

# # call Tesseract
# recognizedTxt = pytesseract.image_to_string(pil_im, lang='eng', config="-psm 13")
# print recognizedTxt


#pil_im.show()

# # remove non-digits
# allchars = string.maketrans('','')
# nodigs = allchars.translate(allchars, string.digits)
# print recognizedTxt.translate(allchars, nodigs)

# cv2.imshow("edged", edged)
# cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 3)
# cv2.imshow("digit tag", img)
# cv2.imshow("cropped", cropped)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


#======= PREDICTION USING TRAINED MODEL ==============


import scipy
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves import cPickle as pickle

# load the model from disk
model_filename = "sklearn_model_01.sav"
loaded_model = pickle.load(open(model_filename, 'rb'))

# constants
pixel_depth = 255.0
tupleDim = (28, 28)

# preprocess image
cropped_gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
# binarization using Otsu method
_, cropped_binarized = cv2.threshold(cropped_gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
#
cropped_resized = cv2.resize(cropped_binarized, tupleDim, interpolation = cv2.INTER_LINEAR)
## display binarized image
#cv2.imshow("cb", cropped_resized)
#cv2.waitKey(0)
#cv2.destroyAllWindows()
#print type(cropped_resized)
#print cropped_resized.shape

# run prediction
img_data = (cropped_resized.astype(float) - pixel_depth / 2) / pixel_depth
img_height, img_width = img_data.shape
ax_2D = img_data.reshape( (1, img_height*img_width) )
#print(ax_2D.shape)    
predictions =  loaded_model.predict(ax_2D)
print(filepath, predictions[0])